### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [ ]:
%pip install langchain

In [4]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [ ]:
!pip install langchain_groq

In [52]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama3-70b-8192", groq_api_key=groq_api_key,       
temperature=1,
max_tokens=1024
)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BCBD514220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BCBD516770>, model_name='llama3-70b-8192', temperature=1.0, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=1024)

In [ ]:
!pip install langchain_core

In [68]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to bangla"),
    HumanMessage(content="so youre saying i'm not capable enough for you ? you're damn wrong")
]

result=model.invoke(messages)

In [69]:
result

AIMessage(content='তাহলে আপনি বলছেন আমি আপনার জন্য যথেষ্ট সক্ষম নই ? আপনি খুব ভুল আছেন।', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 38, 'total_tokens': 124, 'completion_time': 0.250485868, 'prompt_time': 0.00478435, 'queue_time': 0.022482429, 'total_time': 0.255270218}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4bc89a1-7b05-435f-858a-da8ead7bc376-0', usage_metadata={'input_tokens': 38, 'output_tokens': 86, 'total_tokens': 124})

In [70]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'তাহলে আপনি বলছেন আমি আপনার জন্য যথেষ্ট সক্ষম নই ? আপনি খুব ভুল আছেন।'

In [77]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'তাহলে আপনি বলছেন আমি আপনার জন্য যথেষ্ট নই? আপনি খুবই ভুল!'

In [78]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [79]:
result=prompt.invoke({"language":"bangla","text":"so youre saying i'm not capable enough for you ? you're damn wrong"})

In [80]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into bangla:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="so youre saying i'm not capable enough for you ? you're damn wrong", additional_kwargs={}, response_metadata={})]

In [81]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"bangla","text":"so youre saying i'm not capable enough for you ? you're damn wrong"})

'তাহলে আপনি বলছেন আমি আপনার জন্য যথেষ্ট নই? আপনি খুব ভুল!'

In [ ]:
!pip install streamlit